In [1]:
# 기본적인 데이터 핸들링
import pandas as pd
import numpy as np
from datetime import datetime

# Crawling
import requests
from bs4 import BeautifulSoup

# 시간 제어 및 진행 경과 확인
from tqdm.notebook import tqdm
import time
import random

In [2]:
# Naver 증권의 뉴스공시 페이지에서 제목, 링크, 게시일을 수집
def get_title_link_date(code_page):
    # 네이버 증권 뉴스공시 페이지에서 종목과 페이지를 제외한 링크
    front_url = f"https://finance.naver.com/item/news_news.naver?"
    end_url = "&sm=title_entity_id.basic&clusterId="
    
    # User-Agent를 통해서 봇 의심 우회
    headers = {"User-Agent": 
               "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    
    # 데이터 저장을 위한 리스트 생성
    stock_items, titles, links, dates = [], [], [], []
    
    # 진행률 확인을 위한 tqdm 생성
    outer_bar = tqdm(code_page.items(), total=len(code_page.items()), position=0)
    
    # 입력받은 종목 딕셔너리와 페이지 수 리스트를 통해서 제목, 링크, 게시일 수집
    for c_key, c_item in outer_bar:
        outer_bar.set_description(f'Porcessiong Stock {c_key}')
        
        pages = [i for i in range(1,c_item[1]+1)]
        inner_bar = tqdm(pages, total=len(pages), leave=False, position=1)
        # 키워드는 고정된 채로 페이지를 넘기며 수집
        for p in inner_bar:
            inner_bar.set_description(f'Porcessiong Page {p}')
            
            query = f"code={c_item[0]}&page={p}"
            url = front_url + query + end_url
            response = requests.get(url, headers=headers)
            bs_obj = BeautifulSoup(response.text, 'html.parser')

            title_link_tag = bs_obj.find_all('td', {'class' : 'title'})
            date_tag = bs_obj.find_all('td', {'class' : 'date'})
            # 해당 페이지에서 스크랩핑한 정보들 저장
            for i in range(len(title_link_tag)):
                title = title_link_tag[i].find('a').text
                link = 'https://finance.naver.com'+title_link_tag[i].find('a')['href']
                date = datetime.strptime(date_tag[i].text[1:11],'%Y.%m.%d').date()
                
                # 수집한 제목, 링크, 게시일과 해당 뉴스의 종목을 리스트에 저장
                stock_items.append(c_key)
                titles.append(title)
                links.append(link)
                dates.append(date)
                
            time.sleep(0.5 + random.random())
        time.sleep(0.5 + random.random())
        inner_bar.close()
    outer_bar.close()
    # 수집한 데이터를 데이터 프레임으로 저장
    df_crawling = pd.DataFrame({'Stock_Item' : stock_items, 'Title' : titles, 'Link' : links, 'Date' : dates})
    
    return df_crawling

In [3]:
# 2023년의 데이터만 수집하기 위해서 각 기업과 종목코드, 페이지번호가 담긴 딕셔너리 생성
dict_semiconductor = {'삼성전자' : ['005930',401], 'SK하이닉스' : ['000660',153]}

dict_electric_car_battery = {'현대차' : ['005380',401], 'LG에너지솔루션' : ['373220',111]}

dict_petrochemistry = {'LG화학' : ['051910',64], 'SK이노베이션' : ['096770',35],
                  '롯데케미칼' : ['011170',41], 'S-OIL' : ['010950',38]}

dict_shipbuilding_shiping = {'HD현대중공업' : ['329180',19], '대우조선해양' : ['042660',51], '현대미포조선' : ['010620',11],
                             '삼성중공업' : ['010140',15], 'HD한국조선해양' : ['009540',44], 'HMM' : ['011200',24],
                             '팬오션' : ['028670',3]}

dict_steel = {'현대제철' : ['004020',28], 'POSCO홀딩스' : ['005490',215], '고려아연' : ['010130',16]}

In [4]:
# get_title_link_date 함수를 사용해서 섹터별 데이터 프레임 생성
df_semiconductor = get_title_link_date(dict_semiconductor)

df_electric_car_battery = get_title_link_date(dict_electric_car_battery)

df_petrochemistry = get_title_link_date(dict_petrochemistry)

df_shipbuilding_shiping = get_title_link_date(dict_shipbuilding_shiping)

df_steel = get_title_link_date(dict_steel)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/401 [00:00<?, ?it/s]

  0%|          | 0/153 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/401 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/215 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [5]:
display(df_semiconductor, df_electric_car_battery, df_petrochemistry, df_shipbuilding_shiping, df_steel)

,Stock_Item,Title,Link,Date
0,삼성전자,[유안타證 주간추천주]삼성전자·두산·POSCO홀딩스,https://finance.naver.com/item/news_read.naver...,2023-06-04
1,삼성전자,“5만원짜리 17만원에 샀어요” 삼성 ‘이 제품’ 뭐길래 못 사서 난리...,https://finance.naver.com/item/news_read.naver...,2023-06-03
2,삼성전자,이러다 현대차도 제칠라...‘베트남의 삼성’이 아메리칸 드림 꿈 꿔보지...,https://finance.naver.com/item/news_read.naver...,2023-06-03
3,삼성전자,“6월 삼성전자·카카오 뜬다”...반도체·인터넷 업종 강세 전망,https://finance.naver.com/item/news_read.naver...,2023-06-03
4,삼성전자,“이래도 애플페이 써야 돼?” 수수료 중국 5배 폭리…오히려 무료 결단...,https://finance.naver.com/item/news_read.naver...,2023-06-03
...,...,...,...,...
13359,SK하이닉스,"""연말 대목 효과 없었다""…SK하이닉스 이어 삼성까지 적자 예고",https://finance.naver.com/item/news_read.naver...,2022-12-29
13360,SK하이닉스,"삼성전자, 폐장일 5만6000원 붕괴…하이닉스 신저가",https://finance.naver.com/item/news_read.naver...,2022-12-29
13361,SK하이닉스,[특징주] 올해 마지막 거래일에 삼성전자·SK하이닉스 1%대 하락세…미...,https://finance.naver.com/item/news_read.naver...,2022-12-29
13362,SK하이닉스,"SK하이닉스, 4분기 적자 전망에...연일 '52주 신저가'",https://finance.naver.com/item/news_read.naver...,2022-12-29


,Stock_Item,Title,Link,Date
0,현대차,이러다 현대차도 제칠라...‘베트남의 삼성’이 아메리칸 드림 꿈 꿔보지...,https://finance.naver.com/item/news_read.naver...,2023-06-03
1,현대차,"美서 돈 덜쓰고 더 많이 판 현대차…""제품력 개선, 펀더멘털 달라졌다""",https://finance.naver.com/item/news_read.naver...,2023-06-03
2,현대차,“중고차 시장에선 내가 잘나가”…현대 아반떼·미니 컨트리맨 주목,https://finance.naver.com/item/news_read.naver...,2023-06-03
3,현대차,"현대차 노조, ""40년 근속자에 금메달 달라"" 外[금주의 산업계 이슈]",https://finance.naver.com/item/news_read.naver...,2023-06-03
4,현대차,현대차는 ‘세단’·기아는 ‘SUV’…1~5월 완성차 효자는?,https://finance.naver.com/item/news_read.naver...,2023-06-03
...,...,...,...,...
11238,LG에너지솔루션,"LG엔솔, 4Q 실적 컨센서스 하회 '숨고르기'-한국",https://finance.naver.com/item/news_read.naver...,2023-01-02
11239,LG에너지솔루션,제2의 LG엔솔 나올까…“새해 IPO 침체 지속 전망”,https://finance.naver.com/item/news_read.naver...,2023-01-02
11240,LG에너지솔루션,코스피 기관 순매도 1위 'LG에너지솔루션',https://finance.naver.com/item/news_read.naver...,2023-01-01
11241,LG에너지솔루션,'단군 이래 최대어' LG엔솔에 가려진 IPO 시장 그림자,https://finance.naver.com/item/news_read.naver...,2022-12-30


,Stock_Item,Title,Link,Date
0,LG화학,"LG화학, 450만 달러 투자한 기술 반환…항암제 역량 집중 시동",https://finance.naver.com/item/news_read.naver...,2023-06-02
1,LG화학,"신학철 LG화학 부회장, 일본서 ‘인재 확보’",https://finance.naver.com/item/news_read.naver...,2023-06-02
2,LG화학,신학철 “LG화학 일원 돼 달라” 日 인재들에 러브콜,https://finance.naver.com/item/news_read.naver...,2023-06-02
3,LG화학,"""LG화학에서 꿈 펼치라""…신학철 부회장, 日 인재에 '손짓'",https://finance.naver.com/item/news_read.naver...,2023-06-02
4,LG화학,"""인재 확보"" 신학철 LG화학 부회장, 일본까지 날아갔다",https://finance.naver.com/item/news_read.naver...,2023-06-02
...,...,...,...,...
4385,S-OIL,"류열 에쓰오일 사장, ‘CES 2023’ 현장 참관…“전략 고민 기회...",https://finance.naver.com/item/news_read.naver...,2023-01-06
4386,S-OIL,"류열 S-OIL 사장, CES 참관…신사업 분야 투자기업 격려",https://finance.naver.com/item/news_read.naver...,2023-01-06
4387,S-OIL,원유 정제 마진 바닥 쳤다…에쓰오일 추천[베스트 애널리스트 추천 종목],https://finance.naver.com/item/news_read.naver...,2023-01-04
4388,S-OIL,"울산시, ‘현대차·에쓰오일’ 전담팀 만들어 투자 기업 지원",https://finance.naver.com/item/news_read.naver...,2022-12-30


,Stock_Item,Title,Link,Date
0,HD현대중공업,[유가증권시장 공시] HD현대중공업 / HSD엔진 / 한신공영 등,https://finance.naver.com/item/news_read.naver...,2023-06-01
1,HD현대중공업,"HD현대중공업, 1조2392억 규모 컨테이너선 5척 수주[주목 e공시]",https://finance.naver.com/item/news_read.naver...,2023-06-01
2,HD현대중공업,"[공시] HD현대중공업, 1조2400억 규모 컨테이너선 공사 수주",https://finance.naver.com/item/news_read.naver...,2023-06-01
3,HD현대중공업,"HD현대중공업, 1조2392억원 규모 컨테이너선 공사 수주",https://finance.naver.com/item/news_read.naver...,2023-06-01
4,HD현대중공업,"HD현대중공업, 1조2392억 규모 컨테이너선 5척 수주 계약",https://finance.naver.com/item/news_read.naver...,2023-06-01
...,...,...,...,...
4136,팬오션,"팬오션, 한진칼 지분 5% 취득… “단순 투자 목적”",https://finance.naver.com/item/news_read.naver...,2022-12-06
4137,팬오션,"팬오션, 한진칼 지분 5% 취득",https://finance.naver.com/item/news_read.naver...,2022-12-06
4138,팬오션,"하림지주 ""팬오션, 한진칼 지분 5.8% 취득…1259억원 규모""",https://finance.naver.com/item/news_read.naver...,2022-12-06
4139,팬오션,"팬오션, 호반건설 보유 한진칼 지분 매입",https://finance.naver.com/item/news_read.naver...,2022-12-06


,Stock_Item,Title,Link,Date
0,현대제철,"현대제철, 창립 70주년 걸음기부 캠페인…1억원 기금 조성 목표",https://finance.naver.com/item/news_read.naver...,2023-06-02
1,현대제철,"현대제철, 아동 이동체어 걸음 기부 캠페인 펼쳐",https://finance.naver.com/item/news_read.naver...,2023-06-02
2,현대제철,'하루 5000보의 나눔'..창립 70년 현대제철 걸음기부 캠페인,https://finance.naver.com/item/news_read.naver...,2023-06-02
3,현대제철,"현대제철, '걸음 기부' 문샷 챌린지 진행…1억원 기부 목표",https://finance.naver.com/item/news_read.naver...,2023-06-02
4,현대제철,"현대제철, ‘걸음 기부 캠페인’ 문샷 챌린지 진행…1억원 기부 목표",https://finance.naver.com/item/news_read.naver...,2023-06-02
...,...,...,...,...
6796,고려아연,울산 고려아연 바둑팀 창단…KB국민은행 바둑리그 참가,https://finance.naver.com/item/news_read.naver...,2022-12-22
6797,고려아연,"고려아연, 사랑의 열매 ‘희망나눔캠페인 300억 클럽’ 가입",https://finance.naver.com/item/news_read.naver...,2022-12-20
6798,고려아연,"고려아연, 사랑의열매에 이웃돕기 성금 30억원 기탁",https://finance.naver.com/item/news_read.naver...,2022-12-20
6799,고려아연,"고려아연, 연말 맞아 ‘이웃사랑 성금’ 30억 기탁",https://finance.naver.com/item/news_read.naver...,2022-12-20


In [ ]:
# 수집한 데이터 csv로 저장
df_semiconductor.to_csv('Title_Link_Date/df_semiconductor.csv', index=False)
df_electric_car_battery.to_csv('Title_Link_Date/df_electric_car_battery.csv', index=False)
df_petrochemistry.to_csv('Title_Link_Date/df_petrochemistry.csv', index=False)
df_shipbuilding_shiping.to_csv('Title_Link_Date/df_shipbuilding_shiping.csv', index=False)
df_steel.to_csv('Title_Link_Date/df_steel.csv', index=False)